In [1]:
# Code #2 (reads master_df.xlsx, which is generated from "Capstone_HL_TargetY.ipynb", and generates "master_df_with_features.xlsx" as output)
# This code generates features using the existing OEP enrollment data file ONLY i.e. no ADI/Census data appended yet)
# OEP PUFs (2023-2025) are read to generate churn metrics (market newness, passive renewal, net attrition)
# Other features are generated using latest OEP PUF (2025).
# master_df_features is the main dataframe and final output that is saved as "output/"master_df_with_features.xlsx"

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
OEP_2025_demographic = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(5) by Demographics')
OEP_2025_metal = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(6) by Metal Level')
OEP_2025_FPL = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(7) by FPL')
OEP_2025_FA = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(4) by Premium and FA')

OEP_2025_enrollment = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(1) by Enrollment Status') 
OEP_2024_enrollment = pd.read_excel('data/2024 OEP County-Level Public Use File.xlsx', sheet_name='(1) by Enrollment Status')
OEP_2023_enrollment = pd.read_excel('data/2023 OEP County-Level Public Use File.xlsx', sheet_name='(1) by Enrollment Status')

master_df = pd.read_excel('output/master_df_v2.xlsx')

In [3]:
#---------------------------------------------------------
#CLEAN FIPS for OEP Enrollment Data Files (to calculate churn rate between OEP years)
#---------------------------------------------------------
def clean_fips(df):
    df.columns = df.columns.str.strip()
    # Handle common FIPS column variations
    fips_col = None
    if 'County FIPS Code' in df.columns:
        fips_col = 'County FIPS Code'
    elif 'FIPS' in df.columns:
        fips_col = 'FIPS'
    
    if fips_col:
        # Rename to standard
        df.rename(columns={fips_col: 'County FIPS Code'}, inplace=True)
        df['County FIPS Code'] = df['County FIPS Code'].astype(str).str.zfill(5)
    return df

def clean_cms_numeric(series):
    """
    Handles CMS suppression logic:
    - Replaces '*' (1-10 people) with 0 for conservative targeting.
    - Coerces other non-numeric strings to NaN, then fills with 0.
    """
    # Replace the suppression character explicitly
    series = series.replace('*', 0)
    # Convert to numeric, turning any other errors into NaN
    return pd.to_numeric(series, errors='coerce').fillna(0)


# Clean standard files
dfs = [OEP_2025_demographic, OEP_2025_metal, OEP_2025_FA, OEP_2025_FPL, master_df]
for df in dfs:
    df = clean_fips(df)

OEP_2025_enrollment = clean_fips(OEP_2025_enrollment)
OEP_2024_enrollment = clean_fips(OEP_2024_enrollment)
OEP_2023_enrollment = clean_fips(OEP_2023_enrollment)


In [4]:
OEP_2025_metal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158 entries, 0 to 2157
Data columns (total 9 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   State                                                2158 non-null   object 
 1   County FIPS Code                                     2158 non-null   object 
 2   County Name                                          2157 non-null   object 
 3   Number of Consumers with an Exchange Plan Selection  2157 non-null   float64
 4   Catastrophic                                         2157 non-null   object 
 5   Bronze                                               2157 non-null   object 
 6   Silver                                               2157 non-null   object 
 7   Gold                                                 2157 non-null   object 
 8   Platinum                                             2157 non-null   

In [5]:
#---------------------------------------------------------
#FEATURE ENGINEERING
#---------------------------------------------------------

# --- A. FINANCIAL VULNERABILITY FEATURES ---
fa_features = OEP_2025_FA.copy()
fa_features = clean_fips(fa_features)
fa_features = fa_features.rename(columns={
    'Average Premium': 'avg_premium_before_APTC',
    'Average Premium after APTC': 'avg_premium_after_APTC',
    'Average APTC among Consumers Receiving APTC': 'avg_aptc_val',
    'Consumers with a Premium ≤$10 after APTC': 'count_le_10',
    'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_fa'
})

cols_to_num = ['avg_premium_before_APTC', 'avg_aptc_val', 'count_le_10', 'total_enrollees_fa', 'avg_premium_after_APTC']
for c in cols_to_num:
    #fa_features[c] = pd.to_numeric(fa_features[c], errors='coerce').fillna(0) ## previous code to convert NAs to 0's; custom function clean_cms_numeric specifically handles * cells and should be better
    fa_features[c] = clean_cms_numeric(fa_features[c])

# FEATURE 1: Subsidy Reliance Ratio
fa_features['feat_subsidy_reliance'] = fa_features['avg_aptc_val'] / fa_features['avg_premium_before_APTC'].replace(0, 1)
# FEATURE 2: % of enrollees paying less than $10 monthly premium after APTC
fa_features['feat_pct_paying_under_10'] = (fa_features['count_le_10'] / fa_features['total_enrollees_fa'].replace(0, 1)) * 100
fa_features['feat_subsidy_coverage_ratio'] = (fa_features['avg_premium_before_APTC'] - fa_features['avg_premium_after_APTC']) / fa_features['avg_premium_before_APTC']

# --- B. INCOME FEATURES ---
FPL_features = OEP_2025_FPL.copy()
FPL_features = clean_fips(FPL_features)
FPL_map = {
    '≥100% to ≤150% of FPL': 'count_FPL_100_150',
    '>150% to ≤200% of FPL': 'count_FPL_150_200',
    '>200% to ≤250% of FPL': 'count_FPL_200_250',
    '>250% to ≤300% of FPL': 'count_FPL_250_300',
    '>300% to ≤400% of FPL': 'count_FPL_300_400',
    '>400% of FPL': 'count_FPL_400_plus',
    'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_FPL'
}

FPL_features = FPL_features.rename(columns=FPL_map)
FPL_cols = list(FPL_map.values())

for f in FPL_cols:
    if f in FPL_features.columns:
        FPL_features[f] = clean_cms_numeric(FPL_features[f])

for fcol in FPL_cols[:-1]: 
    feat_name = 'feat_pct_' + fcol.replace('count_', '')
    FPL_features[feat_name] = (FPL_features[fcol] / FPL_features['total_enrollees_FPL'].replace(0, 1)) * 100

# FEATURE 3: FPL %
FPL_features['feat_pct_FPL_150_250'] = FPL_features['feat_pct_FPL_150_200'] + FPL_features['feat_pct_FPL_200_250']
FPL_features['feat_pct_FPL_250_400'] = FPL_features['feat_pct_FPL_250_300'] + FPL_features['feat_pct_FPL_300_400']
FPL_features = FPL_features.drop(columns=['feat_pct_FPL_150_200', 'feat_pct_FPL_200_250','feat_pct_FPL_250_300','feat_pct_FPL_300_400'])

# --- C. PLAN MIX FEATURES ---
metal_features = OEP_2025_metal.copy()
metal_features = clean_fips(metal_features)
metal_features = metal_features.rename(columns={
    'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_metal'
})

metals = ['Bronze', 'Silver', 'Gold', 'Platinum', 'total_enrollees_metal']
for m in metals:
    if m in metal_features.columns:
        #metal_features[m] = pd.to_numeric(metal_features[m], errors='coerce').fillna(0) ## previous code to convert NAs to 0's; custom function clean_cms_numeric specifically handles * cells and should be better
        metal_features[m] = clean_cms_numeric(metal_features[m])
    else:
        metal_features[m] = 0

#metal_features['total_enrollees_metal'] = metal_features[metals].sum(axis=1)

# FEATURE 4: Metal Tier %
metal_features['feat_pct_silver'] = (metal_features['Silver'] / metal_features['total_enrollees_metal'].replace(0, 1)) * 100
metal_features['feat_pct_bronze'] = (metal_features['Bronze'] / metal_features['total_enrollees_metal'].replace(0, 1)) * 100
metal_features['feat_pct_gold'] = (metal_features['Gold'] / metal_features['total_enrollees_metal'].replace(0, 1)) * 100
metal_features['feat_pct_premium_tier'] = ((metal_features['Gold'] + metal_features['Platinum']) / metal_features['total_enrollees_metal'].replace(0, 1)) * 100
metal_features['plan_value_density'] = metal_features['feat_pct_gold'] / (metal_features['feat_pct_bronze'] + metal_features['feat_pct_gold']) #creates a single metric that captures consumer sentiment (e.g. when enrollee steps away from silver middle, do they go gold or bronze?). Often cleaner for K-Means than having two separate dimension col (i.e. feat_pct_gold and feat_pct_bronze).


# --- D. DEMOGRAPHIC FEATURES ---
demo_features = OEP_2025_demographic.copy()
demo_features = clean_fips(demo_features)
demo_map = {
    'Black / African American, Non-Hispanic': 'count_black',
    'Hispanic': 'count_hispanic',
    'Unknown Hispanic': 'count_unknown_hispanic',
    'White, Non-Hispanic': 'count_white',
    'Asian, Non-Hispanic': 'count_asian',
    'American Indian / Alaska Native, Non-Hispanic': 'count_aian',
    'Native Hawaiian / Pacific Islander, Non-Hispanic': 'count_nhpi',
    'Multiracial, Non-Hispanic': 'count_multiracial',
    'Other Race, Non-Hispanic': 'count_other_race',
    'Unknown Race, Non-Hispanic': 'count_unknown_race',
    'Female': 'count_female',
    'Age 18-25': 'count_age_18_25',
    'Age 26-34': 'count_age_26_34',
    'Age 55-64': 'count_age_55_64',
    'Age ≥ 65': 'count_age_65_over',
    'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_demo'
}

demo_features = demo_features.rename(columns=demo_map)
demo_cols = list(demo_map.values())

for c in demo_cols:
    if c in demo_features.columns:
        #demo_features[c] = pd.to_numeric(demo_features[c], errors='coerce').fillna(0) ## previous code to convert NAs to 0's; custom function clean_cms_numeric specifically handles * cells and should be better
        demo_features[c] = clean_cms_numeric(demo_features[c])
    else:
        demo_features[c] = 0

# Feature 5: Race/Ethnicity and Age composition (Calculated as %)
# Calculate Percentages
for col in demo_cols[:-1]: 
    feat_name = 'feat_pct_' + col.replace('count_', '')
    demo_features[feat_name] = (demo_features[col] / demo_features['total_enrollees_demo'].replace(0, 1)) * 100

demo_features['feat_pct_age_18_34'] = demo_features['feat_pct_age_18_25'] + demo_features['feat_pct_age_26_34']
demo_features['feat_pct_age_55_over'] = demo_features['feat_pct_age_55_64'] + demo_features['feat_pct_age_65_over']
demo_features = demo_features.drop(columns=['feat_pct_age_18_25', 'feat_pct_age_26_34','feat_pct_age_55_64','feat_pct_age_65_over'])

# Feature 6: Gender
demo_features['feat_pct_female'] = (demo_features['count_female'] / demo_features['total_enrollees_demo'].replace(0, 1)) * 100


# Feature 7: STABILITY FEATURES (Renewals, Market newness, Net attrition) ---
stability_features = pd.DataFrame()

enroll_25 = OEP_2025_enrollment.copy()
enroll_24 = OEP_2024_enrollment.copy()
enroll_23 = OEP_2023_enrollment.copy()

col_map_25 = {
    'Number of Consumers with an Exchange Plan Selection': 'Total',
    'New Consumers': 'New',
    'Total Re-enrollees': 'ReEnrollees',
    'Active Re-enrollees': 'Active',
    'Automatic Re-enrollees': 'Automatic'
    }

col_map_24 = {
    'Number of Consumers with a Marketplace Plan Selection': 'Total',
    'New Consumers': 'New',
    'Total Re-enrollees': 'ReEnrollees',
    'Active Re-enrollees': 'Active',
    'Automatic Re-enrollees': 'Automatic'
    }

enroll_25 = enroll_25.rename(columns=col_map_25)
enroll_24 = enroll_24.rename(columns=col_map_24)
enroll_23 = enroll_23.rename(columns=col_map_24)

# Clean Numerics
for df_enr in [enroll_25, enroll_24, enroll_23]:
        for c in ['Total', 'New', 'ReEnrollees', 'Active', 'Automatic']:
            if c in df_enr.columns: df_enr[c] = clean_cms_numeric(df_enr[c])

# 1. Metric B: Market Newness (Instability)
# Formula: New / Total (%)
enroll_25['feat_market_newness'] = (enroll_25['New'] / enroll_25['Total'].replace(0, 1)) * 100
    
# 2. Metric C: Passive Renewal Risk
# Formula: Automatic / (Active + Automatic) (%)
enroll_25['feat_passive_renewal'] = (enroll_25['Automatic'] / (enroll_25['Active'] + enroll_25['Automatic']).replace(0, 1)) * 100
    
# 3. Metric A: Net Attrition (Historical)
# Formula: 1 - (ReEnrollees / Total) -- calculate for both years (2023-2024, 2024-2025) then take the average
# Calculate Year 1 Attrition (2024 -> 2025)
att_1 = pd.merge(enroll_24[['County FIPS Code', 'Total']], 
                 enroll_25[['County FIPS Code', 'ReEnrollees']], 
                 on='County FIPS Code', suffixes=('_24', '_25'))
att_1['retention_rate'] = att_1['ReEnrollees'] / att_1['Total'].replace(0, 1)
att_1['attrition_recent'] = (1 - att_1['retention_rate']) * 100
att_1['attrition_recent'] = att_1['attrition_recent'].clip(lower=0)
 
# Calculate Year 2 Attrition (2023 -> 2024)
att_2 = pd.merge(enroll_23[['County FIPS Code', 'Total']], 
                 enroll_24[['County FIPS Code', 'ReEnrollees']], 
                 on='County FIPS Code', suffixes=('_23', '_24'))
att_2['retention_rate'] = att_2['ReEnrollees'] / att_2['Total'].replace(0, 1)
att_2['attrition_hist'] = (1 - att_2['retention_rate']) * 100
att_2['attrition_hist'] = att_2['attrition_hist'].clip(lower=0)

# Merge and Average
combined_attrition = pd.merge(att_1[['County FIPS Code', 'attrition_recent']], 
                              att_2[['County FIPS Code', 'attrition_hist']], 
                              on='County FIPS Code', how='outer')

combined_attrition['feat_net_attrition'] = (combined_attrition['attrition_recent'] + combined_attrition['attrition_hist']) / 2

# Combine all features
stability_features = pd.merge(enroll_25[['County FIPS Code', 'feat_market_newness', 'feat_passive_renewal']],
                              combined_attrition[['County FIPS Code', 'feat_net_attrition']],
                              on='County FIPS Code', how='outer')

C:\Users\liuh102\AppData\Local\Temp\ipykernel_20740\1054066677.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('*', 0)
C:\Users\liuh102\AppData\Local\Temp\ipykernel_20740\1054066677.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('*', 0)
C:\Users\liuh102\AppData\Local\Temp\ipykernel_20740\1054066677.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(

In [6]:
# ---------------------------------------------------------
# 4. MERGE EVERYTHING INTO MASTER_DF
# ---------------------------------------------------------
# Create a copy to preserve master_df
master_df_features = master_df.copy()

# Define subsets
fa_subset = fa_features[['County FIPS Code', 'feat_subsidy_reliance', 'feat_pct_paying_under_10', 'avg_premium_before_APTC','avg_premium_after_APTC','feat_subsidy_coverage_ratio']]
FPL_subset = FPL_features[['County FIPS Code', 'feat_pct_FPL_100_150', 'feat_pct_FPL_150_250', 'feat_pct_FPL_250_400', 'feat_pct_FPL_400_plus']]
metal_subset = metal_features[['County FIPS Code', 'feat_pct_gold', 'feat_pct_silver', 'feat_pct_bronze', 'feat_pct_premium_tier','plan_value_density']]
demo_subset = demo_features[['County FIPS Code'] + [c for c in demo_features.columns if c.startswith('feat_pct_')]]

# Sequential Merges onto the features dataframe
master_df_features = pd.merge(master_df_features, fa_subset, on='County FIPS Code', how='left')
master_df_features = pd.merge(master_df_features, metal_subset, on='County FIPS Code', how='left')
master_df_features = pd.merge(master_df_features, demo_subset, on='County FIPS Code', how='left')
master_df_features = pd.merge(master_df_features, FPL_subset, on='County FIPS Code', how='left')


if not stability_features.empty:
    master_df_features = pd.merge(master_df_features, stability_features, on='County FIPS Code', how='left')

# Fill NaN with 0
master_df_features = master_df_features.fillna(0)


In [7]:
# ---------------------------------------------------------
# 5. PREVIEW
# ---------------------------------------------------------
print("\n--- Feature Engineering Complete (Triad + Historical Smoothing) ---")
if not stability_features.empty:
    print("Top 5 Counties with Highest Passive Renewal (Bill Shock Risk):")
    print(master_df_features[['State', 'County FIPS Code', 'feat_passive_renewal', 'Access_Disruption_Risk_Index_RMS']]
          .sort_values(by='feat_passive_renewal', ascending=False).head(5))

    print("\nTop 5 Counties with Highest Avg Net Attrition (Shrinking Markets):")
    print(master_df_features[['State', 'County FIPS Code', 'feat_net_attrition', 'Access_Disruption_Risk_Index_RMS']]
          .sort_values(by='feat_net_attrition', ascending=False).head(5))

master_df_features.to_excel('output/master_df_with_features.xlsx', index=False)
print("\nSaved to: master_df_with_features.xlsx")


--- Feature Engineering Complete (Triad + Historical Smoothing) ---
Top 5 Counties with Highest Passive Renewal (Bill Shock Risk):
               State County FIPS Code  feat_passive_renewal  \
1933           Texas            48421             82.426778   
1739           Texas            48033             82.275711   
1194  North Carolina            37065             77.209302   
118         Arkansas            05013             76.795580   
443          Indiana            18161             76.562500   

      Access_Disruption_Risk_Index_RMS  
1933                         67.695696  
1739                         59.815698  
1194                         63.559770  
118                          50.231691  
443                          60.497311  

Top 5 Counties with Highest Avg Net Attrition (Shrinking Markets):
         State County FIPS Code  feat_net_attrition  \
1037   Montana            30069               100.0   
1857     Texas            48269               100.0   
67      Al

In [11]:
master_df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156 entries, 0 to 2155
Data columns (total 50 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        2156 non-null   int64  
 1   County FIPS Code                  2156 non-null   object 
 2   State                             2156 non-null   object 
 3   total_enrollees_master            2156 non-null   int64  
 4   count_age_55_64                   2156 non-null   int64  
 5   count_age_65_plus                 2156 non-null   int64  
 6   count_gt_400_fpl                  2156 non-null   int64  
 7   count_gt_100_150_fpl              2156 non-null   int64  
 8   count_gt_150_200_fpl              2156 non-null   int64  
 9   benchmark_60_401_diff             2156 non-null   float64
 10  count_55_plus                     2156 non-null   int64  
 11  factor_age_55_plus                2156 non-null   float64
 12  V1_low